In [2]:
from tqdm import tqdm
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
from datetime import date
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
import time



In [3]:
# Define the calculate_sum function
def calculate_sum(row):
    total = 0
    for key, val in values.items():
        if row[key] not in bad_list:
            total += val
    return total

In [6]:
from tqdm import tqdm
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
from datetime import date
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

import time

# Define the calculate_sum function
def calculate_sum(row):
    total = 0
    for key, val in values.items():
        if row[key] not in bad_list:
            total += val
    return total

start_time = time.time()

# Create the engine 
engine = create_engine('mysql+mysqlconnector://' + 'bob' + ':' + '0akcabin3t' + '@' + 'localhost' + ':' + '3307' + '/imi', echo=False)

entry_run_time = 'Q3_072023'
entry_state = 'Virginia'
entry_city = ''

# Dynamic query construction
conditions = []
if entry_run_time:
    conditions.append(f'i.run_time = "{entry_run_time}"')
if entry_state:
    conditions.append(f'i.state = "{entry_state}"')
if entry_city:
    conditions.append(f'i.city_fix = "{entry_city}"')
condition_str = ' AND '.join(conditions)

query_kitems = '''
SELECT 
    i.identifier,
    i.run_time,
    i.imb_name,
    i.address,
    i.city_fix,
    i.state,
    i.zip,
    i.county_fix,
    i.is_erotic,
    i.permanently_closed,
    i.phone,
    i.url,
    r.message1,
    r.message2,
    r.review_head_date,
    r.finger,
    r.kiss,
    r.lick,
    r.fs,
    r.breast_play,
    r.blowjob,
    r.ass_play,
    r.hand_job,
    r.prostate_massage,
    r.paid_to_house,
    r.extra_tip
FROM imi.reviews r
LEFT JOIN imi.imb i 
    ON i.identifier = r.imb_identifier
''' + (' WHERE ' + condition_str) if condition_str else ''

df_kitems = pd.read_sql(query_kitems, engine)


#df_kitems = pd.read_excel('/Users/emmabarnes/Documents/some more xlsx/reviews_3year_random.xlsx')
df_kitems.head(50)


# Load original DataFrame here (df_kitems)

# Perform necessary conversions outside the loop
today = date.today()
value05 = today - datetime.timedelta(days=365)


# Create new columns to store calculated values


df_kitems['date_interpolation'] = np.nan
df_kitems['value_illicit'] = np.nan
df_kitems['final_value'] = np.nan
df_kitems['IMB_label'] = np.nan

# Define values and bad_list as you did in your code
values = {
    'finger': 0.075,
    'kiss': 0.025,
    'lick': 0.075,
    'breast_play': 0.025, 
    'blowjob': 0.35,
    'ass_play': 0.025,
    'hand_job': 0.35,
    'prostate_massage': 0.075
}
bad_list = set(['X', 'x', '', '?'])

illicit_threshold = 0.3

# Iterate through unique business IDs
for business_id in tqdm(df_kitems['identifier'].unique()):
    
    entry = len(df_kitems[df_kitems['identifier']==business_id])
    
    mask = df_kitems['identifier'] == business_id
    temporal_df = df_kitems[mask]
    
    list_dates = temporal_df['review_head_date']
    list_dates_diff = (today - list_dates).dt.days


    xp = [0, (today - value05).days, (today - list_dates.min()).days]
    fp = [1, 0.5, 0]
    interpolations = np.interp(list_dates_diff, xp, fp)

    value_illicit = temporal_df.apply(calculate_sum, axis=1)
    
    df_kitems.loc[mask, 'date_interpolation'] = interpolations
    df_kitems.loc[mask, 'value_illicit'] = value_illicit
    df_kitems.loc[mask, 'final_value'] = interpolations * value_illicit
    
    temporal_df.loc[mask, 'date_interpolation'] = interpolations
    temporal_df.loc[mask, 'value_illicit'] = value_illicit
    temporal_df.loc[mask, 'final_value'] = interpolations * value_illicit
        
    exit = len(temporal_df)
    if entry == exit:        

        sum_date_interpolation = temporal_df['date_interpolation'].sum()
        sum_final_value = temporal_df['final_value'].sum()

         # Calculate IMB_label and classify as illicit or not_illicit

        if sum_date_interpolation != 0:
            imb_label = sum_final_value / sum_date_interpolation
            df_kitems.loc[mask, 'IMB_label'] = imb_label
        
            # Label as illicit or not_illicit based on threshold
            if imb_label >= illicit_threshold:
                df_kitems.loc[mask, 'Activity_Label'] = 'illicit'
            elif imb_label <= 0.35 and imb_label >= 0.25:
                df_kitems.loc[mask,'Activity_Label'] = 'questionable_illicit'
            else:
                df_kitems.loc[mask, 'Activity_Label'] = 'not_illicit'
        else:
            df_kitems.loc[mask, 'IMB_label'] = np.nan
            df_kitems.loc[mask, 'Activity_Label'] = np.nan
        
        
#thisfinal
#whatveer you want
print (time.time() - start_time, 's')
df_kitems.head(50)

#df_kitems.to_excel('/Users/emmabarnes/Documents/some more xlsx/reviews_3year_random_illicit_class.xlsx')


100%|██████████| 623/623 [00:02<00:00, 257.49it/s]

5.779993772506714 s


,identifier,run_time,imb_name,address,city_fix,state,zip,county_fix,is_erotic,permanently_closed,phone,url,message1,message2,review_head_date,finger,kiss,lick,fs,breast_play,blowjob,ass_play,hand_job,prostate_massage,paid_to_house,extra_tip,date_interpolation,value_illicit,final_value,IMB_label,Activity_Label
0,Q3_072023Bliss Massage434-293-2951406 E Main S...,Q3_072023,Bliss Massage,"406 E Main St Charlottesville, Virginia 22902",Charlottesville,Virginia,22902,Charlottesville City,0,0,434-293-2951,/erotic-massage-bliss-massage-charlottesville-...,Passable Massage for price but appears to be n...,"Seemed like was going to be erotic, while mass...",2014-12-18,X,X,X,X,X,X,X,X,X,30,10,0.000000,0.000,0.000000,NaN,NaN
1,Q3_072023Happy Feet massage757-698-4224701-M N...,Q3_072023,Happy Feet massage,"701-M N Battlefield Blvd, Chesapeake, VA 23320...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-698-4224,/erotic-massage-happy-feet-massage-chesapeake-...,Legit good massage if you have sore muscles.,Standard Massage there were no extras provided...,2023-04-13,X,X,X,X,X,X,X,X,X,50,20,0.763014,0.000,0.000000,0.000000,not_illicit
2,Q3_072023Happy Feet massage757-698-4224701-M N...,Q3_072023,Happy Feet massage,"701-M N Battlefield Blvd, Chesapeake, VA 23320...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-698-4224,/erotic-massage-happy-feet-massage-chesapeake-...,"The place has a very modern look to it, almost...","Cici is a thin, cute and looks young, but migh...",2021-12-20,X,X,X,X,X,X,X,X,X,50,10,0.424074,0.000,0.000000,0.000000,not_illicit
3,Q3_072023Happy Feet massage757-698-4224701-M N...,Q3_072023,Happy Feet massage,"701-M N Battlefield Blvd, Chesapeake, VA 23320...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-698-4224,/erotic-massage-happy-feet-massage-chesapeake-...,So I was on craigslist browsing trying to find...,Overall the massage was good the therapist was...,2017-07-31,X,X,X,X,X,X,X,X,X,40.00,30,0.000000,0.000,0.000000,0.000000,not_illicit
4,Q3_072023Molihua Massage757-818-9177801 Volvo ...,Q3_072023,Molihua Massage,"801 Volvo Parkway, #141 Chesapeake, Virginia 2...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-818-9177,/erotic-massage-molihua-massage-chesapeake-va-...,2 girls there one short thick and other tall s...,Every time I come here the 2 girls literally b...,2022-11-26,o,o,X,CFS,o,X,o,o,o,60,150,0.573973,0.575,0.330034,0.575000,illicit
5,Q3_072023Massage Elite757-277-9995200 N Battle...,Q3_072023,Massage Elite,"200 N Battlefield Blvd #9 Chesapeake, Virginia...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-277-9995,/erotic-massage-massage-elite-chesapeake-va-12453,Sarah is a very nice Chinese lady. I believe s...,So Sarah isn't a bombshell but she'd be a very...,2022-10-17,X,X,X,X,X,X,X,X,X,60,20,0.519178,0.000,0.000000,0.303602,illicit
6,Q3_072023Massage Elite757-277-9995200 N Battle...,Q3_072023,Massage Elite,"200 N Battlefield Blvd #9 Chesapeake, Virginia...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-277-9995,/erotic-massage-massage-elite-chesapeake-va-12453,"There were 3 girls working when I arrived, for...","I went for the first time, I pass by it often ...",2022-04-21,X,X,X,X,X,X,X,X,X,60,20,0.477660,0.000,0.000000,0.303602,illicit
7,Q3_072023Massage Elite757-277-9995200 N Battle...,Q3_072023,Massage Elite,"200 N Battlefield Blvd #9 Chesapeake, Virginia...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-277-9995,/erotic-massage-massage-elite-chesapeake-va-12453,"Sarah can work out those kinks in a great way,...",This time around I was looking for a regular m...,2021-09-25,X,X,X,X,X,X,X,X,X,60,10,0.449499,0.000,0.000000,0.303602,illicit
8,Q3_072023Massage Elite757-277-9995200 N Battle...,Q3_072023,Massage Elite,"200 N Battlefield Blvd #9 Chesapeake, Virginia...",Chesapeake,Virginia,23320,Chesapeake City,0,0,757-277-9995,/erotic-massage-massage-elite-chesapeake-va-12453,NO EXTRAS! Great massage but disappointing end...,I was ready to leave an amazing review for the...,2021-02-1

In [9]:
temporal_df['review_head_date']

12020    2022-07-25
12021    2022-04-26
12022    2021-09-16
12023    2021-05-27
12024    2021-03-22
12025    2021-01-22
12026    2020-09-06
12027    2020-08-27
12028    2020-06-22
12029    2019-10-11
12030    2019-10-01
12031    2019-09-02
12032    2019-08-16
12033    2019-08-08
12034    2019-08-01
12035    2019-07-19
12036    2019-05-20
12037    2019-04-21
12038    2018-11-14
12039    2018-10-18
12040    2018-10-18
12041    2018-10-14
12042    2018-08-06
12043    2018-07-12
12044    2018-05-08
12045    2018-04-11
12046    2018-02-26
12047    2018-01-06
12048    2017-11-10
12049    2017-09-23
12050    2017-09-19
12051    2017-08-02
12052    2017-05-30
12053    2017-05-22
12054    2017-03-29
12055    2017-02-22
12056    2016-12-24
12057    2016-12-20
12058    2016-10-27
12059    2016-09-12
12060    2016-09-06
12061    2016-07-21
12062    2016-07-05
12063    2016-04-19
12064    2016-03-01
12065    2016-02-28
12066    2016-02-07
12067    2015-12-29
12068    2015-12-17
12069    2015-11-16


In [8]:
temporal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 12020 to 12075
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   identifier          56 non-null     object 
 1   run_time            56 non-null     object 
 2   imb_name            56 non-null     object 
 3   address             56 non-null     object 
 4   city_fix            56 non-null     object 
 5   state               56 non-null     object 
 6   zip                 56 non-null     object 
 7   county_fix          56 non-null     object 
 8   is_erotic           56 non-null     int64  
 9   permanently_closed  56 non-null     int64  
 10  phone               56 non-null     object 
 11  url                 56 non-null     object 
 12  message1            56 non-null     object 
 13  message2            56 non-null     object 
 14  review_head_date    56 non-null     object 
 15  finger              56 non-null     object 
 16  kis